
# 🕺🕺🕺 Follow Your Pose 💃💃💃: 
# Pose-Guided Text-to-Video Generation using Pose-Free Videos

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi - -query-gpu = name, memory.total, memory.free - -format = csv, noheader
#make sure you are using Tesla T4, 15360 MiB, 15101 MiB


# 🕺🕺🕺 Install Environment

In [ ]:
# @title  Environment Setup
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

!git clone https: // github.com/jiaxinlarhh/ict3104-team13-2023.git

!git clone https: // github.com/open-mmlab/mmpose.git


In [ ]:
# @title  Setup FollowYourPose & MMPose

# FollowYourPose
!cd / content/ict3104-team13-2023
!export PYTHONPATH = /content/ict3104-team13-2023: $PYTHONPATH
!python - m pip install - q - U - -pre triton
!apt update
!python - m pip install - q diffusers == 0.11.1 torch == 1.13.1 transformers == 4.26.0 bitsandbytes == 0.35.4 imageio-ffmpeg xformers == 0.0.16 - -extra-index-url https: // download.pytorch.org/whl/cu113

# MMPose
%cd / content/mmpose
!python3 - m pip install torch torchvision torchaudio - -index-url https: // download.pytorch.org/whl/cu118
# install MMEngine, MMCV and MMDetection using MIM
!python3 - m pip install - U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"
!mim install "mmdet>=3.0.0"


# 🕺🕺🕺 Data Exploration

In [ ]:
#@title  US T13-4 Load Video
import ipywidgets as widgets
import os, cv2
from os import listdir
from google.colab.patches import cv2_imshow
from IPython.display import  HTML
from base64 import b64encode

# variables
data_url = None
vid_directory = "./ict3104-team13-2023/videos"
vid_list = []

# store video names in list
for files in os.listdir(vid_directory):
  if files[0] != ".":
    vid_list.append(files)

# show vid name in list as dropdown
dropdown = widgets.Dropdown(options=vid_list, value=None)

# UI
button = widgets.Button(description="Enter")
output = widgets.Output()
display(dropdown, button, output)

# UI functions
def on_button_clicked(b):
    with output:
        mp4 = open(vid_directory+'/'+dropdown.value,'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        if mp4 and data_url:
          display(HTML("""
                  <video controls>
                        <source src="%s" type="video/mp4">
                  </video>
                  """ % data_url))
        else:
          print("error opening vid file")


button.on_click(on_button_clicked)

# 🕺🕺🕺 Inference

Due to memory of GPU, we recommend set video_length=8 in ./config/pose_sample.yaml for running successfully. 

Meanwhile, we should keep the skeleton frame length(./followyourpose/pipelines/pipeline_followyourpose.py:422 ) equal with video_length

In [ ]:
%cd /content/FollowYourPose
!pwd
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"

# 🕺🕺🕺 Training

In [ ]:
#@title  User Story T13-6
import ipywidgets as widgets
import os, cv2
import subprocess
import yaml
from os import listdir
from google.colab.patches import cv2_imshow
from IPython.display import HTML, clear_output
from base64 import b64encode


# Define and Instantiate variables
charades_data_url = None
charades_video_path = "./charades"
charades_video_list = []
chosen_charades_video = None

# Store names of charades video in a list
for file in os.listdir(charades_video_path):
  charades_video_list.append(file)

# Show input to accept user prompt
prompt_input = widgets.Text(
    value='',  # Initial value
    placeholder='Enter prompt...',  # Placeholder text
    description='Prompt Input: ',  # Label for the input
)

# Add names of charades video as dropdown options
charades_videos_dropdown = widgets.Dropdown(options=charades_video_list, value=None)

# UI to show after running this cell
choose_charades_video_button = widgets.Button(description="Choose Video")
chosen_video_output = widgets.Output()

# Display all UI
display(prompt_input, charades_videos_dropdown, choose_charades_video_button, chosen_video_output)

# method to add user's prompt into pose_sample.yaml
def insert_prompt_input_into_config(prompt):

  # Load the YAML file
  with open('./configs/pose_sample.yaml', 'r') as file:
      config = yaml.safe_load(file)

  # Access the 'prompts' section
  config['validation_data']['prompts'] = [prompt]


  # Save the modified configuration back to the file
  with open('./configs/pose_sample.yaml', 'w') as file:
      yaml.dump(config, file, default_flow_style=False)

def generate_gif():
  # Change directory to /content/ict3104-team13-2023
  os.chdir('/content/ict3104-team13-2023')

  # Print the current working directory
  print(os.getcwd())

  # Set the TORCH_DISTRIBUTED_DEBUG environment variable and launch txt2video.py
  subprocess.run(['accelerate', 'launch', 'txt2video.py', '--config=configs/pose_sample.yaml', '--skeleton_path=./pose_example/vis_ikun_pose2.mov'])

#
def set_charades_video_variables(charades_video_name):
  if charades_video_name is not None:
    # print("Have something")
    pass
  with chosen_video_output:
        charades_mp4 = open(charades_video_path +'/'+ charades_video_name,'rb').read()
        charades_data_url = "data:video/mp4;base64," + b64encode(charades_mp4).decode()
        if charades_mp4 and charades_data_url:
          video_html = f'<video controls><source src="{charades_data_url}" type="video/mp4"></video>'
          # Clear previous output
          clear_output()
          display(HTML(video_html))
        else:
          print("Cannot open chosen charades video")

# OnClick function for 'Choose Video' button
def on_choose_charades_video_button_clicked(b):
    pass

# OnChange function for dropdown
def on_charades_videos_dropdown_change(change):
    if change['name'] == 'value' and change['new']:
        chosen_charades_video = change['new']
        selected_option = change['new']
        # print(f"Selected option: {selected_option}")
        set_charades_video_variables(selected_option)

# On Prompt Input 'enter' key press
def on_prompt_input_enter_pressed(change):
      # print("Enter pressed with text:", prompt_input.value)
      insert_prompt_input_into_config(prompt_input.value)
      generate_gif()

# Attach event functions to UI
prompt_input.on_submit(on_prompt_input_enter_pressed)
charades_videos_dropdown.observe(on_charades_videos_dropdown_change, names='value')
choose_charades_video_button.on_click(on_choose_charades_video_button_clicked)

# 🕺🕺🕺 Testing

# 💃💃💃 The videos are output into ./output/samples